In [1]:
!pip install beautifulsoup4

In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

In [ ]:
def get_links(page_number):
    URL = "https://ielts69.com/essay/p/" + str(page_number)
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, "html.parser")
    results = soup.find(class_="similar-post-list")
    link_elements = results.find_all("a", href=True)
    
    links = []
    
    for link_element in link_elements:
        link_url = link_element["href"]
        links.append(link_url)
    
    return links

In [ ]:
all_links = []
for i in range(1,2061):
    links = get_links(i)
    all_links.append(links)

all_links


In [ ]:
list_of_links = np.asarray(all_links).ravel()

In [ ]:
df = pd.DataFrame(list_of_links)
df.to_csv("essay_links.csv")

In [3]:
links = pd.read_csv("essay_links.csv")
list_of_links = links['0'].to_numpy()

In [4]:
def get_data(page_url):
    new_dict = {}

    URL2 = 'https://ielts69.com' + str(page_url)
    essay_page = requests.get(URL2)
    soup2 = BeautifulSoup(essay_page.content, "html.parser")

    essay_text = soup2.find("div", id="editor")
    #text = results2.find_all()
    
    if essay_text != None:
        for tag in essay_text.select('div.popup'):
            tag.decompose()
    
        delimiter = '#'      

        for line_break in essay_text.findAll('br'):      
            line_break.replaceWith(delimiter)

    paragraph_number = soup2.find("i", id="totalParagraphs")
    word_count = soup2.find("i", id="totalWords")
    essay_topic = soup2.find("h1")
    essay_score = soup2.find("div", id="bandTotal")

    ccTotal = soup2.find("i", id="ccTotal")
    lrTotal = soup2.find("i", id="lrTotal")
    graTotal = soup2.find("i", id="graTotal")
    taTotal = soup2.find("i", id="taTotal")

    new_dict = {'EssayTopic': essay_topic.text, 'EssayBody': "No text" if essay_text == None else essay_text.text, 
            'ParagraphCount': int(paragraph_number.text), 'WordCount': int(word_count.text), 
            'EssayScore': float(essay_score.text), 'CC_Score': float(ccTotal.text), 
            'LR_Score': float(lrTotal.text), 'GRA_Score': float(graTotal.text), 'TA_Score': float(taTotal.text)}
    
    return new_dict

In [5]:
Essay_Dataset = pd.DataFrame(columns=['EssayTopic', 'EssayBody', 'ParagraphCount', 
                                     'WordCount', 'EssayScore', 'CC_Score', 'LR_Score', 
                                     'GRA_Score', 'TA_Score'])

In [6]:
complete = 0
for index_number, url in enumerate(list_of_links[:4]):
    data = get_data(url)
    new_row = pd.DataFrame(data, index=[index_number])
    Essay_Dataset = pd.concat([Essay_Dataset, new_row])
    
    if ((index_number % 60) == 0) and (index_number != 0):
        complete += 1
        print("%{} is compl11ete".format(complete))
        
print("Done!")

Done!


In [7]:
Essay_Dataset.to_csv("essay_dataset_test.csv")

In [8]:
Essay_Dataset

,EssayTopic,EssayBody,ParagraphCount,WordCount,EssayScore,CC_Score,LR_Score,GRA_Score,TA_Score
0,Increasing the price of petrol is the best way...,"Over the last century, our cities faced unprec...",4,350,6.0,5.5,5.5,6.5,6.5
1,Fatherhood ought to be emphasized as much as m...,"According to some people, the role of a father...",4,419,6.0,5.5,5.5,6.5,7.0
2,Best way to reduce the number of crime among y...,"Thesedays, number of crime commit by the young...",3,197,5.0,5.5,5.0,6.5,5.0
3,Some people believe that it is good to share a...,Some people believe that sharing details of re...,4,281,6.0,6.5,5.5,6.5,6.0
